# Troon Intelligence Agent - ML Models

**Training 3 Machine Learning Models for Golf Course Intelligence**

1. **CHURN_RISK_PREDICTOR** - Predicts member churn risk
2. **NO_SHOW_PREDICTOR** - Predicts probability of tee time no-show
3. **UPGRADE_CANDIDATE_PREDICTOR** - Identifies members for tier upgrades

## Prerequisites
- Database: `TROON_INTELLIGENCE`
- Schema: `ML_MODELS`
- Warehouse: `TROON_WH`
- Packages: `snowflake-ml-python`, `scikit-learn`, `pandas`

In [ ]:
# Import required libraries
from snowflake.snowpark import Session
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.linear_model import LogisticRegression
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.registry import Registry
import warnings

warnings.filterwarnings('ignore')

# Get current session
session = Session.builder.getOrCreate()
session.use_database("TROON_INTELLIGENCE")
session.use_schema("ML_MODELS")
session.use_warehouse("TROON_WH")

print("✅ Session configured")

In [ ]:
# Initialize Registry
registry = Registry(
    session=session,
    database_name="TROON_INTELLIGENCE",
    schema_name="ML_MODELS"
)
print("✅ Registry Initialized")

## Model 1: Churn Risk Predictor
Predicts if a member is at risk of leaving based on tenure, spend, and play frequency.

In [ ]:
# Load Data
df_churn = session.table("TROON_INTELLIGENCE.ANALYTICS.V_CHURN_RISK_FEATURES")
train_churn, test_churn = df_churn.random_split([0.8, 0.2], seed=42)

# Drop ID columns not needed for training
train_churn = train_churn.drop("MEMBER_ID")
test_churn = test_churn.drop("MEMBER_ID")

# Define Pipeline
churn_pipeline = Pipeline([
    ("Classifier", RandomForestClassifier(
        label_cols=["CHURN_RISK_LABEL"],
        output_cols=["PREDICTED_RISK"],
        n_estimators=10,
        max_depth=5
    ))
])

# Train
churn_pipeline.fit(train_churn)
print("✅ Churn Model Trained")

# Register
try:
    registry.delete_model("CHURN_RISK_PREDICTOR")
except:
    pass

registry.log_model(
    model=churn_pipeline,
    model_name="CHURN_RISK_PREDICTOR",
    sample_input_data=train_churn.drop("CHURN_RISK_LABEL").limit(10),
    comment="Predicts churn risk (0=Low, 1=Med, 2=High)"
)
print("✅ Churn Model Registered")

## Model 2: No-Show Predictor
Predicts probability of a booking being a No-Show.

In [ ]:
# Load Data
df_noshow = session.table("TROON_INTELLIGENCE.ANALYTICS.V_NO_SHOW_FEATURES")
train_noshow, test_noshow = df_noshow.random_split([0.8, 0.2], seed=42)

# Drop ID columns not needed for training
train_noshow = train_noshow.drop("BOOKING_ID")
test_noshow = test_noshow.drop("BOOKING_ID")

# Define Pipeline
noshow_pipeline = Pipeline([
    ("Classifier", LogisticRegression(
        label_cols=["NO_SHOW_LABEL"],
        output_cols=["NO_SHOW_LABEL"],
        max_iter=100
    ))
])

# Train
noshow_pipeline.fit(train_noshow)
print("✅ No-Show Model Trained")

# Register
try:
    registry.delete_model("NO_SHOW_PREDICTOR")
except:
    pass

registry.log_model(
    model=noshow_pipeline,
    model_name="NO_SHOW_PREDICTOR",
    sample_input_data=train_noshow.drop("NO_SHOW_LABEL").limit(10),
    comment="Predicts if booking will be No Show (1) or Completed (0)"
)
print("✅ No-Show Model Registered")

## Model 3: Upgrade Candidate Predictor
Identifies members suitable for tier upgrades.

In [ ]:
# Load Data
df_upgrade = session.table("TROON_INTELLIGENCE.ANALYTICS.V_UPGRADE_FEATURES")
train_upgrade, test_upgrade = df_upgrade.random_split([0.8, 0.2], seed=42)

# Drop ID columns not needed for training
train_upgrade = train_upgrade.drop("MEMBER_ID")
test_upgrade = test_upgrade.drop("MEMBER_ID")

# Define Pipeline
upgrade_pipeline = Pipeline([
    ("Classifier", RandomForestClassifier(
        label_cols=["UPGRADE_CANDIDATE_LABEL"],
        output_cols=["UPGRADE_CANDIDATE_LABEL"],
        n_estimators=10,
        max_depth=5
    ))
])

# Train
upgrade_pipeline.fit(train_upgrade)
print("✅ Upgrade Model Trained")

# Register
try:
    registry.delete_model("UPGRADE_CANDIDATE_PREDICTOR")
except:
    pass

registry.log_model(
    model=upgrade_pipeline,
    model_name="UPGRADE_CANDIDATE_PREDICTOR",
    sample_input_data=train_upgrade.drop("UPGRADE_CANDIDATE_LABEL").limit(10),
    comment="Predicts if member is candidate for upgrade (1) or not (0)"
)
print("✅ Upgrade Model Registered")